In [ ]:
import os
import glob
import numpy as np
import tensorflow as tf
import tensorflow.keras.layers as tkl
from PIL import Image
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
import keras
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))


In [ ]:
from unet_attention import build_attention_unet

In [ ]:

folder_path_train = r'C:\Users\monar\Python_projects\ISRO\trainx2'
folder_path_trainmask  = r'C:\Users\monar\Python_projects\ISRO\trainy2'

train_images = np.zeros((4932,256,256,3))
train_masks = np.zeros((4932,256,256))

for k in range(1,138):
    
    train_images[(k-1)*36:k*36] =  np.load(os.path.join(folder_path_train,f"{k}.npy"),mmap_mode='r')
    train_masks[(k-1)*36:k*36] =   np.load(os.path.join(folder_path_trainmask,f"{k}.npy"),mmap_mode='r')[:,:,:,0]
    

In [ ]:

train_masks = (train_masks > 0.5 ).astype(np.uint8)


In [ ]:
from matplotlib import pyplot as plt
print(train_images[0].dtype)
train_images = train_images.astype(np.float16)

In [ ]:
model = build_attention_unet((256,256,3),1)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

In [ ]:
checkpoint = ModelCheckpoint('unet_segmentation.keras', verbose=1, save_best_only=True)

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, verbose=1)

early_stopping = EarlyStopping(monitor='val_loss', patience=5, verbose=1)

In [ ]:
import gc
gc.collect()


In [ ]:
from sklearn.model_selection import train_test_split


x,x_val,y,y_val = train_test_split(train_images,train_masks,test_size=0.3)

In [ ]:
model.fit(x=x,y=y,batch_size=8,validation_split=0.1,epochs=10,verbose=2,callbacks=[checkpoint,early_stopping, reduce_lr])
